In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("rdd_api").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/01 14:55:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/01 14:55:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
sc = spark.sparkContext

## List of Distinct Genres

In [11]:
def save_all_genres():
    movies_rdd = spark.sparkContext.textFile('Dataset/movies.dat')
    genres_rdd = movies_rdd.map(lambda x: x.split("::")[2])
    genres_rdd = genres_rdd.flatMap(lambda x: x.split("|"))
    distinct_genres_rdd = genres_rdd.distinct().sortBy(lambda x: x)
    return distinct_genres_rdd.saveAsTextFile("processed_data/distinct_genres")

In [12]:
save_all_genres()

/Users/ankitrajvanshi/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better " "support with spilling")
/Users/ankitrajvanshi/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better " "support with spilling")


In [14]:
def get_all_genres():
    data = sc.textFile("processed_data/distinct_genres").collect()
    return data

In [15]:
print(get_all_genres())

['Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir']


In [16]:
def save_list_of_years():
    movies_rdd = spark.sparkContext.textFile('Dataset/movies.dat')
    title_year_rdd = movies_rdd.map(lambda x: x.split("::")[1])
    year_rdd = title_year_rdd.map(lambda x: x[-5:-1])
    distinct_years_rdd = year_rdd.distinct().sortBy(lambda x: x)
    return distinct_years_rdd.saveAsTextFile("processed_data/list_of_years")

In [17]:
save_list_of_years()

Py4JJavaError: An error occurred while calling o159.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/ankitrajvanshi/MS CE/Sem 3/Big Data/Project/MovilensDatasetAnalytics/movies.dat
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.ap

In [11]:
def save_list_of_starting_letters():
    movies_rdd = spark.sparkContext.textFile('movies.dat')
    title_year_rdd = movies_rdd.map(lambda x: x.split("::")[1])
    first_char_rdd = title_year_rdd.map(lambda x: x[0])
    distinct_first_char_rdd = first_char_rdd.distinct().sortBy(lambda x: x)
    return distinct_first_char_rdd.collect()

In [12]:
save_list_of_starting_letters()

['$', "'", '.', '1', '2', '3', '4', '5', '7', '8', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'e']
